In [54]:
from pprint import pprint
import pandas as pd
import nltk
import regex as re

Разделение перечислений на отдельные навыки

In [55]:
not_to_split = ['a/b', 'ci/cd', 'pl/sql', 'pl/pg', 'tcp/ip', 'ui/ux', 'ux/ui', 'а/б', 'а/в', 'в/из']

In [56]:
def split_skills(row):
    skills_dict_temp = []
    for skill in row:
        skill = skill.lower()
        res = [el for el in not_to_split if(el in skill)]
        if bool(res) == False:
            if re.search('[/\,(]', skill) == None:
                skills_dict_temp.append(skill)
            else:
                splited_skill = re.split('[/\,(]', skill)
                for skill in splited_skill:
                    skills_dict_temp.append(skill)
        else:
            skills_dict_temp.append(skill)
    return skills_dict_temp


Удаление нерелевантных символов

In [57]:
def clean_text(skill):
    skill = re.sub(r'1c', r'1с', skill)
    skill = re.sub(r'1[\s][сc]', r'1с', skill)
    if re.search('1с', skill) == None:
        skill = re.sub(r'[0-9]', '', skill)
    else:
        #skill = re.sub('[0]', '', skill)
        skill = re.sub('0|[2-9]', '', skill)
    res = [el for el in not_to_split if(el in skill)]
    if bool(res) == False:
        skill = re.sub(r'[^\w\s+]', ' ', skill).strip()
    skill = re.sub(r'\s+', ' ', skill)
    return skill

Токенизация слов

In [58]:
from nltk import word_tokenize

Лемматизация

In [59]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [60]:
def lemmatization(word_list):
    lemm_word_list = []
    for word in word_list:
        lemm_word_list.append(morph.parse(word)[0].normal_form)
    return lemm_word_list

Стемминг

In [61]:
from nltk.stem import SnowballStemmer
snowball = SnowballStemmer(language='russian')

In [62]:
def stemming(word_list):
    stemmed = []
    for word in word_list:
        stemmed.append(snowball.stem(word))
    return stemmed

Удаление стоп слов

In [63]:
from nltk.corpus import stopwords

In [64]:
sw_to_add = ['хорошее', 'знание', 'знать', 'опыт', 'работы', 'организация', 'работать', 'ms', 'условия', 'умение', 'навык']
sw_to_add = lemmatization(sw_to_add)
stopwords = stopwords.words('russian')
stopwords = stopwords + sw_to_add

In [65]:
def remove_stop_words(tokens):
    tokens = [word for word in tokens if word not in stopwords]
    return tokens

Составление n-грамм

In [66]:
from nltk.util import ngrams

In [67]:
def most_frequent_ngrams(ngrams_column, n_most_common):
    ngrams_list = ngrams_column.to_list()
    ngrams = []
    for row in ngrams_list:
        for ngram in row:
            ngrams.append(ngram)
    frequence = nltk.FreqDist(ngrams)
    return frequence.most_common(n_most_common)           

Получение данных о скилах во всех вакансиях

In [68]:
check = pd.read_json('deduplicated.json')['key_skills']
skills = []
for element in check:
    if len(element) > 0:
        skills = skills + element
print(len(skills))
#182538

93110


In [69]:
skills = split_skills(skills)

In [70]:
dff = pd.DataFrame(skills)

In [71]:
dff['orig'] = dff[0]
dff = dff.drop(columns = 0)

In [72]:
dff['clean'] = dff['orig'].apply(lambda x: clean_text(x))

Не учитываем 1с, для 1с отдельно

In [73]:
dff = dff.loc[dff['clean'].str.contains('1с') == False]

In [74]:
dff['tokenize'] = dff['clean'].apply(lambda x: word_tokenize(x, language='russian'))

In [75]:
dff['lemmatize'] = dff['tokenize'].apply(lambda x: lemmatization(x))

In [76]:
dff['lemmatize'] = dff['lemmatize'].apply(lambda x: remove_stop_words(x))

In [77]:
dff['bigrams'] = dff['lemmatize'].apply(lambda x: list(ngrams(x, 2)))

In [78]:
dff['trigrams'] = dff['lemmatize'].apply(lambda x: list(ngrams(x, 3)))

In [130]:
most_common = most_frequent_ngrams(dff['lemmatize'], 301)

In [131]:
most_common_bigrams = most_frequent_ngrams(dff['bigrams'], 177)

In [132]:
most_common_trigrams = most_frequent_ngrams(dff['trigrams'], 100)

In [133]:
pprint(most_common_trigrams)

[(('больший', 'объём', 'информация'), 1651),
 (('разработка', 'технический', 'задание'), 1436),
 (('постановка', 'задача', 'разработчик'), 990),
 (('моделирование', 'бизнес', 'процесс'), 842),
 (('сбор', 'анализ', 'информация'), 499),
 (('оптимизация', 'бизнес', 'процесс'), 418),
 (('анализ', 'финансовый', 'показатель'), 385),
 (('анализ', 'бизнес', 'показатель'), 373),
 (('анализ', 'текущий', 'финансовый'), 322),
 (('текущий', 'финансовый', 'показатель'), 322),
 (('описание', 'бизнес', 'процесс'), 185),
 (('agile', 'project', 'management'), 183),
 (('анализ', 'ассортиментный', 'линейка'), 172),
 (('продвинуть', 'пользователь', 'пк'), 169),
 (('поиск', 'информация', 'интернет'), 165),
 (('аналитический', 'склад', 'ум'), 150),
 (('анализ', 'инвестиционный', 'проект'), 145),
 (('опытный', 'пользователь', 'пк'), 135),
 (('анализ', 'бизнес', 'процесс'), 132),
 (('анализ', 'конкурентный', 'среда'), 128),
 (('план', 'фактный', 'анализ'), 117),
 (('фактный', 'анализ', 'бюджет'), 117),
 (('кон

In [134]:
pprint(most_common)

[('анализ', 11862),
 ('sql', 5209),
 ('бизнес', 4714),
 ('аналитический', 4361),
 ('мышление', 3705),
 ('данные', 3047),
 ('информация', 2487),
 ('финансовый', 2326),
 ('системный', 2276),
 ('процесс', 2260),
 ('разработка', 2244),
 ('bpmn', 2193),
 ('powerpoint', 2025),
 ('excel', 1932),
 ('технический', 1903),
 ('uml', 1754),
 ('больший', 1737),
 ('аналитика', 1682),
 ('объём', 1672),
 ('задание', 1601),
 ('управление', 1518),
 ('исследование', 1437),
 ('atlassian', 1353),
 ('язык', 1337),
 ('отчётность', 1326),
 ('английский', 1321),
 ('моделирование', 1306),
 ('python', 1297),
 ('проект', 1177),
 ('команда', 1102),
 ('показатель', 1086),
 ('база', 1080),
 ('презентация', 1067),
 ('деловой', 1022),
 ('задача', 1016),
 ('power', 1009),
 ('разработчик', 1005),
 ('постановка', 998),
 ('jira', 977),
 ('пользователь', 934),
 ('bi', 873),
 ('rest', 851),
 ('подготовка', 829),
 ('многозадачность', 813),
 ('visio', 793),
 ('маркетинговый', 745),
 ('рынок', 731),
 ('речь', 730),
 ('грамотный

In [135]:
pprint(most_common_bigrams)

[(('аналитический', 'мышление'), 3063),
 (('бизнес', 'анализ'), 2111),
 (('бизнес', 'процесс'), 1760),
 (('анализ', 'данные'), 1754),
 (('больший', 'объём'), 1669),
 (('объём', 'информация'), 1653),
 (('технический', 'задание'), 1600),
 (('системный', 'анализ'), 1520),
 (('разработка', 'технический'), 1446),
 (('английский', 'язык'), 1307),
 (('база', 'данные'), 1053),
 (('постановка', 'задача'), 995),
 (('задача', 'разработчик'), 990),
 (('управление', 'проект'), 904),
 (('аналитический', 'исследование'), 848),
 (('моделирование', 'бизнес'), 842),
 (('atlassian', 'jira'), 785),
 (('финансовый', 'анализ'), 707),
 (('финансовый', 'показатель'), 707),
 (('power', 'bi'), 698),
 (('пользователь', 'пк'), 647),
 (('подготовка', 'презентация'), 632),
 (('грамотный', 'речь'), 618),
 (('atlassian', 'confluence'), 565),
 (('анализ', 'бизнес'), 506),
 (('сбор', 'анализ'), 501),
 (('анализ', 'информация'), 501),
 (('анализ', 'рынок'), 497),
 (('системный', 'мышление'), 474),
 (('управленческий', '

In [142]:
df_ngrams = pd.concat([pd.DataFrame({'unigrams': most_common}), pd.DataFrame({'bigrams': most_common_bigrams}), pd.DataFrame({'trigrams': most_common_trigrams})], axis = 1)

In [144]:
df_ngrams.to_excel('ngrams.xlsx')

Повторить тоже самое с полученными стопсловами

In [145]:
df_sw = pd.read_excel('ngrams-dict.xlsx')

In [146]:
df_sw

,Unnamed: 0,unigrams,Unnamed: 2,bigrams,Unnamed: 4,trigrams,Unnamed: 6
0,NaN,NaN,для словаря,NaN,для словаря,NaN,для словаря
1,0.0,"('анализ', 11862)",NaN,NaN,"(('аналитический', 'мышление'), 3063)",NaN,"(('больший', 'объём', 'информация'), 1651)"
2,1.0,NaN,"('sql', 5209)",NaN,"(('бизнес', 'анализ'), 2111)",NaN,"(('разработка', 'технический', 'задание'), 1436)"
3,2.0,"('бизнес', 4714)",NaN,"(('бизнес', 'процесс'), 1760)",NaN,NaN,"(('постановка', 'задача', 'разработчик'), 990)"
4,3.0,"('аналитический', 4361)",NaN,NaN,"(('анализ', 'данные'), 1754)",NaN,"(('моделирование', 'бизнес', 'процесс'), 842)"
...,...,...,...,...,...,...,...
297,296.0,"('мероприятие', 52)",NaN,NaN,NaN,NaN,NaN
298,297.0,"('запуск', 52)",NaN,NaN,NaN,NaN,NaN
299,298.0,"('текст', 51)",NaN,NaN,NaN,NaN,NaN
300,299.0,"('юридический', 50)",NaN,NaN,NaN,NaN,NaN


In [ ]:
uni_sw = df_sw['for_uni'].dropna().to_list() + stopwords
bi_sw = df_sw['for_bi'].dropna().to_list() + stopwords

'уж'

'мышление'